Adapted from https://github.com/astudillo20lca/hypercognitive

**Conference Notebook Kernel: `bw25`**

In [1]:
from pathlib import Path
from fs.zipfs import ZipFS

import numpy as np
import pandas as pd

import bw_processing
import bw2calc as bwc

locate the datapackages

In [2]:
datapackges = {path.stem.split('.')[0]:path for path in Path.cwd().glob('*.zip')}

In [3]:
datapackges.keys()

dict_keys(['ipccs', 'mini_biosphere', 'testdb'])

In [4]:
# data package with impact assessment method
ipcc_dp = bw_processing.load_datapackage(ZipFS(datapackges['ipccs']),proxy=True)
# data package with minimal biosphere database
biosphere_dp = bw_processing.load_datapackage(ZipFS(datapackges['mini_biosphere']),proxy=True)
# data package with foreground database
test_dp = bw_processing.load_datapackage(ZipFS(datapackges['testdb']),proxy=True)

In [5]:
metadata_foreground,_ = test_dp.get_resource('testdb_activity_metadata')

In [6]:
metadata_foreground

,Unnamed: 0,location,name,unit,reference product,id
0,0,NaN,a,kilogram,NaN,70148


In [7]:
metadata_biosphere,_ = biosphere_dp.get_resource('mini_biosphere_activity_metadata')

In [8]:
metadata_biosphere

,Unnamed: 0,location,name,unit,reference product,id
0,0,GLO,"Methane, fossil",kilogram,NaN,70147
1,1,GLO,"Carbon dioxide, fossil",kilogram,NaN,70146


In [9]:
name_id_dict = { 
    **metadata_foreground.set_index('name')['id'].to_dict(),
    **metadata_biosphere.set_index('name')['id'].to_dict()
}
name_id_dict

{'a': 70148, 'Methane, fossil': 70147, 'Carbon dioxide, fossil': 70146}

In [10]:
lca = bwc.LCA(demand={name_id_dict['a']:1},data_objs=[ipcc_dp,biosphere_dp,test_dp])
lca.lci()
lca.lcia()

In [11]:
lca.score

39.70000076293945

now imagine we get measurements for the CO2 and CH4 emissions of activity a, they could be formatted as something like this:

In [12]:
emission_measurements = {
('Carbon dioxide, fossil','a'):[20,30.5,10,0],
('Methane, fossil','a'):[0,2,3,4]
}

In [13]:
emission_measurements_df = pd.DataFrame(emission_measurements)
emission_measurements_df

,"Carbon dioxide, fossil","Methane, fossil"
,a,a
0,20.0,0
1,30.5,2
2,10.0,3
3,0.0,4


In [14]:
indices = [(name_id_dict[substance],name_id_dict[activity]) for substance,activity in emission_measurements_df.columns]
indices

[(70146, 70148), (70147, 70148)]

In [15]:
indices_array = np.array(indices, dtype = bw_processing.INDICES_DTYPE)

In [16]:
emissions_iterator = (row for row in emission_measurements_df.values)

lets update the  

In [17]:
overwriter = bw_processing.create_datapackage()

overwriter.add_dynamic_vector(
    matrix="biosphere_matrix",
    interface=emissions_iterator,
    indices_array=indices_array,
)

In [18]:
lca = bwc.LCA(demand={name_id_dict['a']: 1}, data_objs=[ipcc_dp,biosphere_dp,test_dp,overwriter])
lca.lci()
lca.lcia()
lca.score

20.0

In [19]:
for _ in range(len(emission_measurements_df)):
    try:
        next(lca)
        print(f"updated score: {lca.score}")
    except StopIteration:
        print('sequence completed')
        break

updated score: 89.9000015258789
updated score: 99.10000228881836
updated score: 118.80000305175781
sequence completed
